In [ ]:
import nltk
nltk.download("stopwords")

In [ ]:
import re
import docx
from nltk.corpus import stopwords
from collections import Counter

def extract_keywords_from_docx(docx_file):
    doc = docx.Document(docx_file)
    text = " ".join(para.text for para in doc.paragraphs)
    words = re.findall(r'\b\w+\b', text.lower())
    stopwords_set = set(stopwords.words("english"))
    keywords = [word for word in words if word not in stopwords_set and word.isalpha()]
    return Counter(keywords)

if __name__ == "__main__":
    input_docx = "/content/out.docx"      # Replace with the path to your input DOCX file
    keywords_count = 10                 # Number of most common keywords to display
    keyword_counter = extract_keywords_from_docx(input_docx)
    most_common_keywords = keyword_counter.most_common(keywords_count)
    print("Most common keywords:")
    for keyword,count in most_common_keywords:
        print(f"{keyword}: {count} occurrences")


In [ ]:
pip install python-docx PyMuPDF

In [ ]:
import re
import docx
import fitz
from nltk.corpus import stopwords
from collections import Counter

def extract_keywords_from_docx(docx_file):
    doc = docx.Document(docx_file)
    text = " ".join(para.text for para in doc.paragraphs)
    words = re.findall(r'\b\w+\b', text.lower())
    stopwords_set = set(stopwords.words("english"))
    keywords = [word for word in words if word not in stopwords_set and word.isalpha()]
    return Counter(keywords)

def extract_paragraphs_containing_keywords_from_pdf(pdf_file, keywords):
    pdf_document = fitz.open(pdf_file)
    pdf_text = ""
    selected_paragraphs = []

    for page in pdf_document:
        pdf_text += page.get_text()

    paragraphs = re.split(r'\n\s*\n', pdf_text)
    for paragraph in paragraphs:
        for keyword in keywords:
            if keyword in paragraph.lower():
                selected_paragraphs.append(paragraph)
                break

    pdf_document.close()
    return selected_paragraphs

def print_selected_paragraphs(paragraphs):
    print("Selected paragraphs containing most common keywords:")
    for i, paragraph in enumerate(paragraphs, start=1):
        print(f"{i}. {paragraph}\n")

if __name__ == "__main__":
    input_docx = "/content/out.docx"       # Replace with the path to your input DOCX file
    input_pdf = "/content/mv-84_merged.pdf"         # Replace with the path to your input PDF file
    keywords_count = 10                  # Number of most common keywords to display

    keyword_counter = extract_keywords_from_docx(input_docx)
    most_common_keywords = keyword_counter.most_common(keywords_count)
    keywords = [keyword for keyword, _ in most_common_keywords]

    selected_paragraphs = extract_paragraphs_containing_keywords_from_pdf(input_pdf, keywords)
    print_selected_paragraphs(selected_paragraphs)


In [ ]:
!pip install nltk

In [ ]:
nltk.download('punkt')import re
import docx
import fitz
from nltk.corpus import stopwords
from collections import Counter
from nltk.tokenize import sent_tokenize
from nltk.metrics import jaccard_distance
import nltk


def extract_keywords_from_docx(docx_file):
    doc = docx.Document(docx_file)
    text = " ".join(para.text for para in doc.paragraphs)
    words = re.findall(r'\b\w+\b', text.lower())
    stopwords_set = set(stopwords.words("english"))
    keywords = [word for word in words if word not in stopwords_set and word.isalpha()]
    return Counter(keywords)

def extract_important_sentences(paragraph):
    # Tokenize the paragraph into sentences
    sentences = sent_tokenize(paragraph)

    # Sort sentences by length in descending order
    sentences = sorted(sentences, key=len, reverse=True)

    # Return the first two sentences (you can adjust this number)
    return sentences[:2]

def calculate_paragraph_similarity(paragraph1, paragraph2):
    # Tokenize paragraphs into words (excluding stopwords)
    words1 = [word.lower() for word in re.findall(r'\b\w+\b', paragraph1) if word.lower() not in stopwords_set and word.isalpha()]
    words2 = [word.lower() for word in re.findall(r'\b\w+\b', paragraph2) if word.lower() not in stopwords_set and word.isalpha()]

    # Calculate Jaccard similarity between the two paragraphs
    similarity = 1 - jaccard_distance(set(words1), set(words2))
    return similarity

def extract_three_most_important_sentences(paragraphs):
    # Calculate similarity between each pair of paragraphs
    paragraph_scores = {}
    for i in range(len(paragraphs)):
        for j in range(i + 1, len(paragraphs)):
            similarity = calculate_paragraph_similarity(paragraphs[i], paragraphs[j])
            paragraph_scores[(i, j)] = similarity

    # Find the three most similar paragraphs based on the highest similarity score
    sorted_paragraphs = sorted(paragraphs, key=lambda p: sum(paragraph_scores[(i, j)] for i, j in paragraph_scores if i == paragraphs.index(p) or j == paragraphs.index(p)), reverse=True)
    most_similar_paragraphs = sorted_paragraphs[:3]

    # Extract the two most important sentences from the three most similar paragraphs
    important_sentences_list = [extract_important_sentences(paragraph) for paragraph in most_similar_paragraphs]

    return important_sentences_list

if __name__ == "__main__":
    # ... (previous code remains the same)

    # Extract the two most important and similar sentences from three paragraphs
    important_sentences_list = extract_three_most_important_sentences(selected_paragraphs)

    print("Three most important and similar sentences:")
    for i, sentences in enumerate(important_sentences_list, start=1):
        for j, sentence in enumerate(sentences, start=1):
            print(f"{i}. {sentence}")


In [ ]:
def modify_list_for_plea(sentences_list, company_name):
    # Example modifications, you can customize these based on your specific requirements
    additional_statement = f"The contents of the Para are denied in toto and {company_name} is put to strict proof of the same with cogent evidence."

    plea = f"{company_name}'s Written Statement\n\n"
    for i, sentences in enumerate(sentences_list, start=1):
        cleaned_sentences = [sentence.replace("\n", " ") for sentence in sentences]

        # Add the additional statement to every sentence in the paragraph
        modified_sentences = [f"{sentence} {additional_statement}" for sentence in cleaned_sentences]

        # Combine sentences into a single paragraph
        paragraph = " ".join(modified_sentences)

        # Add a phrase to indicate the company's stance
        plea += f"{i}. {paragraph} As representatives of {company_name}, we firmly stand by this statement."

        # Use persuasive language to emphasize the significance of the statement
        plea += f"    It is imperative that this written statement is taken into serious consideration to provide a comprehensive understanding of the circumstances surrounding the case."

        # Express cooperation and commitment to resolving the matter
        plea += "    We are fully committed to cooperating with all relevant parties and authorities to address and rectify the situation promptly."

        # Add a new line for the next plea
        plea += "\n\n"

    # Add a closing statement expressing gratitude for the opportunity to submit the company's written statement
    plea += f"Thank you for considering {company_name}'s written statement, and we look forward to resolving this matter responsibly and expeditiously."

    return plea

# Assuming you already have 'important_sentences_list' as extracted from the previous step
company_name = "ABC Corporation"  # Replace with the name of the company
pleading_statement = modify_list_for_plea(important_sentences_list, company_name)

# Print the modified plea for the company's written statement
print("Modified plea for company's written statement:")
print(pleading_statement)
